In [63]:
!find . -name ".DS_Store" -delete

In [56]:
import numpy as np
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image

from torchvision import transforms

In [61]:
# 1 - 49: Not Defective
# 50 - 61: Defective
class Image_dataset(Dataset):
    def __init__(self, root_dir, transform = None):
        super().__init__()
        self.root_dir = root_dir
        self.list_files = os.listdir(root_dir)

        self.transform = transform
    
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
        ])

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        image_path = self.list_files[index]
        file_path = os.path.join(self.root_dir, image_path)
        image = Image.open(file_path)
        label = "Defective" if int(image_path.split('.jpg')[0]) >= 50 else "Not Defective"

        if self.transform:
            image = self.transform(image)

        return image, label

In [62]:
ds = Image_dataset("Dataset")
dl = DataLoader(ds, batch_size = 5, shuffle = True)
one_image, one_label = next(iter(dl))
one_image, one_label = one_image[0], one_label[0]
plt.imshow(one_image)
plt.title(one_label)
plt.axis("off")
plt.show()

UnidentifiedImageError: cannot identify image file '/Users/mohamedmafaz/Desktop/CountAI/Dataset/.DS_Store'

In [53]:
one_image.shape

torch.Size([720, 1270, 3])

In [54]:
one_label

'Not Defective'